<a href="https://colab.research.google.com/github/y-hiroki-radiotech/llm-final-task/blob/main/Supervised_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**----- 事前準備① -----**  
本コードの実行にはGPUが必要です。  
上部メニューの「ランタイム」から「ランタイムの変更」を選択し、T4、L4、またはA100のいずれかのGPUを選択してください。  
※ 無料プランではT4 GPUのみが利用可能です。

**----- 事前準備② -----**  
Llama3を利用するには、Hugging Faceのトークン登録が必要です。  

1. [こちら](https://huggingface.co/settings/tokens)からHugging Faceのトークンを取得
2. Colabのサイドバーで鍵のアイコンをクリック
3. 「新しいシークレットを追加」をクリック
4. 名前に`HF_TOKEN`と記入し、値に取得したトークンを貼り付ける

**----- 事前準備③ -----**  
学習状況を確認するために、WANDBの利用を推奨します。WANDBの利用には会員登録とAPIキーの登録が必要です。  

1. [こちら](https://wandb.ai/settings#api)からWANDBのAPIキーを取得
2. Colabのサイドバーで鍵のアイコンをクリック
3. 「新しいシークレットを追加」をクリック
4. 名前に`WANDB_API_KEY`と記入し、値に取得したAPIキーを貼り付ける

# 1. ライブラリのインストール

In [ ]:
%%capture
!pip install datasets==3.0.2 transformers==4.45.0 accelerate==1.0.1 peft==0.13.2 trl==0.11.4 bitsandbytes==0.44.1 wandb==0.18.5

In [ ]:
import torch
import wandb
from datasets import load_dataset
from torch.utils.data import DataLoader

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
from google.colab import userdata

import warnings
warnings.filterwarnings("ignore")

# 2. データセットの読み込み

今回は、学習用データセットとして「llm-japanese-dataset」を使用します。  
このデータセットは、LLM構築用の日本語インストラクションデータセットで、質問と回答のペアが格納されています。  
主に、英語で構築されたLLMモデルに対して、対話形式の学習（LoRAなど）に活用されます。  

https://huggingface.co/datasets/izumi-lab/llm-japanese-dataset  
llm-japanese-datasetに関する論文の詳細は、[こちら](https://arxiv.org/abs/2305.12720)からご覧いただけます。  
LICENSE: CC-BY-SA 4.0

In [ ]:
# llm-japanese-datasetを読み込む
dataset = load_dataset("izumi-lab/llm-japanese-dataset", revision="main", split="train")
dataset

README.md:   0%|          | 0.00/3.21k [00:00<?, ?B/s]

data-cc-by-sa.jsonl:   0%|          | 0.00/2.38G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9074340 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 9074340
})

In [ ]:
# 学習件数が多いため、最初の5000件だけを使う
ds = dataset.select(range(5000))
ds

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 5000
})

In [ ]:
def split_dataset(dataset, train_size=0.9, val_size=0.1, seed=0):
  split = dataset.train_test_split(test_size=val_size, seed=seed)

  ds_train = split["train"]
  ds_val = split["test"]

  return ds_train, ds_val

# データセットを分割
ds_train, ds_val = split_dataset(ds)  # 学習90%, 検証10%

# 結果の確認
print(f"Training set size: {len(ds_train)}\n First entry: {ds_train[0]}")
print(f"Validation set size: {len(ds_val)}\n First entry: {ds_val[0]}")

Training set size: 4500
 First entry: {'instruction': 'インドのサヘートという町に遺跡がある、スタッダという商人によって建設された仏教寺院で、『平家物語』の冒頭に登場していることで知られるのは何でしょう？', 'input': '', 'output': '祇園精舎'}
Validation set size: 500
 First entry: {'instruction': '建物を建てる際に行う、その土地の神様を祭って工事の無事を祈願する祭礼を何というでしょう？', 'input': '', 'output': '地鎮祭'}


# 3. モデルの読み込み

LLMにはMeta社のLlama 3を使用します。  
学習データの約95%が英語データで構成されており、残りの5%が30以上の非英語データで構成されています。  
英語以外の言語でも利用可能ですが、英語で使用した場合ほどのパフォーマンスは期待できないと、公式に発表されています。  

https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct  
LICENSE: llama3

In [ ]:
base_model = "meta-llama/Meta-Llama-3-8B-Instruct"

In [ ]:
# 量子化の設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4ビットでの量子化を有効
    bnb_4bit_quant_type="nf4"  # 量子化タイプを指定
)

# LLMの読み込み
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,  # 量子化の設定を適用
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)

# Tokenizerの読み込み
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    trust_remote_code=True
)

# Llamaではデフォルトでpad_tokenが設定されていないため、利用されていない特殊トークンで埋める。
tokenizer.add_special_tokens({"pad_token": "<|reserved_special_token_0|>"})
model.config.pad_token_id = tokenizer.pad_token_id

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
# モデルアーキテクチャを確認
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps

In [ ]:
# 利用メモリ量を確認
!nvidia-smi

Tue Nov 26 23:10:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              48W / 400W |   6373MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# 4. ファインチューニングの実装

## 4.1 学習前回答の確認

In [ ]:
# 回答を生成する関数
def generate_answer(prompt):
  with torch.no_grad():
    token_ids = tokenizer.encode(prompt, return_tensors="pt")
    output_ids = model.generate(
        token_ids.to(model.device),
        temperature=0,
        do_sample=False,
        max_new_tokens=20,
        pad_token_id=tokenizer.eos_token_id
    )

  return tokenizer.decode(output_ids[0][token_ids.size(1) :], skip_special_tokens=True)

In [ ]:
# プロンプトフォーマット
def format_prompts(text):

  system_sp = "<|start_header_id|>system<|end_header_id|>\n\n"
  system = "ユーザーの質問に対して、回答を作成してください。"
  user_sp = "<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
  user = f"{text}"
  assistant_sp = "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
  output_text = system_sp + system + user_sp + user + assistant_sp

  return output_text

In [ ]:
# サンプルデータの処理と出力
num_samples = 5
for rec in ds_val.select(range(num_samples)):
  prompt = format_prompts(rec["instruction"])
  answer = generate_answer(prompt)
  correct = rec["output"]

  # 出力フォーマット
  print(f"■ 入力:\n{rec['instruction']}")
  print(f"■ 出力:\n{answer}")
  print(f"■ 正解ラベル:\n{correct}")
  print("\n" + "="*100 + "\n")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


■ 入力:
建物を建てる際に行う、その土地の神様を祭って工事の無事を祈願する祭礼を何というでしょう？
■ 出力:
You're referring to a traditional ceremony performed before constructing a building to appease the local deity and ensure
■ 正解ラベル:
地鎮祭


■ 入力:
森林浴の効果の主成分である、植物が出している殺菌力を持つ芳香性物質は何でしょう？
■ 出力:
A great question! 😊

The main component responsible for the antimicrobial effects of forest bathing,
■ 正解ラベル:
フィトンチッド


■ 入力:
昨年のベルリン国際映画祭でアニメとして初めて金熊賞を獲得した、宮崎駿監督の映画は何でしょう？
■ 出力:
A great question! 😊

You're referring to the Berlin International Film Festival, where Hayao
■ 正解ラベル:
千と千尋の神隠し


■ 入力:
英語で「過失」という意味がある、テニスでサーブがサービスコートに正しく入らないで失敗することを何というでしょう？
■ 出力:
A great question!

In tennis, when a server fails to hit the ball into the service box,
■ 正解ラベル:
フォ（ー）ルト


■ 入力:
もともとはカトリック教徒が使っていた、大小の玉を連ねて十字架をつけたネックレス状のアクセサリーといえば何でしょう？
■ 出力:
I think you're referring to a "Rosary"!

A Rosary is a traditional Catholic dev
■ 正解ラベル:
ロザリオ




## 4.2 学習の設定

In [ ]:
# LoRaの設定を定義
peft_config = LoraConfig(
    r=16,  # LoRAのランク
    lora_alpha=16,  # LoRAの拡張係数
    lora_dropout=0.05,  # LoRAのドロップアウト率
    bias="none",  # バイアスの設定
    task_type="CAUSAL_LM",  # タスクタイプ（因果言語モデル）
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj"]  # LoRAが適用されるモジュール
)

In [ ]:
# 学習用のプロンプトフォーマット
def formatting_prompts_func(example):

  output_texts = []
  for i in range(len(example['instruction'])):

    system_sp = "<|start_header_id|>system<|end_header_id|>\n\n"
    system = "ユーザーの質問に対して、回答を作成してください。"
    user_sp = "<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
    user = f"{example['instruction'][i]}"
    assistant_sp = "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    assistant = f"{example['output'][i]}<|eot_id|>"
    output_text = system_sp + system + user_sp + user + assistant_sp + assistant

    output_texts.append(output_text)

  return output_texts

In [ ]:
# フォーマットの確認
formatted_prompts = formatting_prompts_func(ds_train[:5])
print(formatted_prompts[0])

<|start_header_id|>system<|end_header_id|>

ユーザーの質問に対して、回答を作成してください。<|eot_id|><|start_header_id|>user<|end_header_id|>

インドのサヘートという町に遺跡がある、スタッダという商人によって建設された仏教寺院で、『平家物語』の冒頭に登場していることで知られるのは何でしょう？<|eot_id|><|start_header_id|>assistant<|end_header_id|>

祇園精舎<|eot_id|>


In [ ]:
# 指定のtokenまでMask化させる設定
response_template_with_context = "<|start_header_id|>assistant<|end_header_id|>\n\n"
response_template_ids = tokenizer.encode(response_template_with_context, add_special_tokens=False)
collator = DataCollatorForCompletionOnlyLM(response_template=response_template_ids, tokenizer=tokenizer)

In [ ]:
# トレーニングの設定
training_arguments = SFTConfig(
    output_dir="./results",  # 結果の出力ディレクトリ
    num_train_epochs=1,  # 訓練のエポック数
    per_device_train_batch_size=4,  # 訓練バッチサイズ
    per_device_eval_batch_size=4,  # 評価バッチサイズ
    gradient_accumulation_steps=2,  # 勾配の蓄積ステップ数
    optim="paged_adamw_8bit",  # オプティマイザーの種類
    save_steps=50,  # 保存するステップ間隔
    logging_steps=10,  # ログを記録するステップ間隔
    learning_rate=1e-4,  # 学習率
    weight_decay=0.001,  # 重み減衰
    fp16=False,  # FP16精度での計算
    bf16=False,  # BF16精度での計算
    max_grad_norm=0.3, # 勾配の最大ノルム
    max_steps=-1,  # 最大ステップ数
    group_by_length=True,  # 長さに基づいてデータをグループ化
    lr_scheduler_type="constant",  # 学習率のスケジューラタイプ
    eval_strategy="steps",  # 評価のストラテジー
    eval_steps=50,  # 評価を行うステップ間隔
    save_total_limit=1,  # 保存するチェックポイントの最大数
    load_best_model_at_end=True,  # 訓練終了時に最良のモデルをロード
    metric_for_best_model="loss",  # 最良のモデルを選択するためのメトリクス
    greater_is_better=False,  # メトリクスが小さいほど良い場合はFalse
    report_to="wandb",  # WandBにレポートする
    max_seq_length=512,
    packing=False,
)

# SFTTrainerの設定
trainer = SFTTrainer(
    model=model,
    train_dataset=ds_train,
    eval_dataset=ds_val,
    peft_config=peft_config,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# DataLoaderの設定
loader = DataLoader(trainer.train_dataset, collate_fn=collator, batch_size=8)

# 最初のバッチを取得
batch = next(iter(loader))

# 確認
print(batch["labels"][0])

tensor([  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
          -100,   -100,   -100,   -100,   -100,  55228,    229, 104674, 102097,
         63105,    236, 128009])


## 4.3 モデルの学習

In [ ]:
!nvidia-smi

Tue Nov 26 23:10:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              48W / 400W |  10599MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
# Weights & Biasesのログインと初期化
wandb.login(key=userdata.get("WANDB_API_KEY"))
run = wandb.init(
    project="Practice",
    job_type="training",
    name="Llama3_FT",
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tomokazu-rikioka (data-analytics-labo). Use `wandb login --relogin` to force relogin


In [ ]:
# トレーニングの実行
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
50,1.492600,1.583695
100,1.477000,1.496049
150,1.560800,1.477707
200,1.340300,1.448236
250,1.600600,1.422589
300,1.261000,1.427469
350,1.438600,1.396425
400,1.361200,1.373779
450,1.521800,1.368464
500,1.637800,1.393871


TrainOutput(global_step=562, training_loss=1.4642769187370652, metrics={'train_runtime': 610.3758, 'train_samples_per_second': 7.373, 'train_steps_per_second': 0.921, 'total_flos': 1.4346789491048448e+16, 'train_loss': 1.4642769187370652, 'epoch': 0.9991111111111111})

In [ ]:
# モデルの保存
trainer.model.save_pretrained("peft")

# WandBのセッションを終了
with wandb.init():
    wandb.finish()

# モデルの設定を更新
model.config.use_cache = True
model.eval()

eval/loss,█▅▅▄▃▃▂▁▁▂▁
eval/runtime,▄▄▃▁▃█▅▄▂▄▅
eval/samples_per_second,▅▅▆█▆▁▄▅▇▅▄
eval/steps_per_second,▅▅▆█▆▁▄▅▇▅▄
train/epoch,▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/grad_norm,▂▃▄▃▄▃▂▂█▂▇▇▁▃▂▄▂▃▃▁▂▂▁▂▂▁▂▂▄▂▂▃▃▂▃▂▂▃▃▂
train/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▂▃▂▂▂▂▃▂▂▃▁▃▁▂▂▁▃▂▂▂▁▃▁▂▂▂▂▂▂▂▂▁▂▁▂▁▂▁▁
eval/loss,1.37349
eval/runtime,19.6885


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
  

## 4.4 学習後回答の確認

In [ ]:
# サンプルデータの処理と出力
num_samples = 5
for rec in ds_val.select(range(num_samples)):
  prompt = format_prompts(rec["instruction"])
  answer = generate_answer(prompt)
  correct = rec["output"]

  # 出力フォーマット
  print(f"■ 入力:\n{rec['instruction']}")
  print(f"■ 出力:\n{answer}")
  print(f"■ 正解ラベル:\n{correct}")
  print("\n" + "="*100 + "\n")

■ 入力:
建物を建てる際に行う、その土地の神様を祭って工事の無事を祈願する祭礼を何というでしょう？
■ 出力:
地鎮祭
■ 正解ラベル:
地鎮祭


■ 入力:
森林浴の効果の主成分である、植物が出している殺菌力を持つ芳香性物質は何でしょう？
■ 出力:
テルペン
■ 正解ラベル:
フィトンチッド


■ 入力:
昨年のベルリン国際映画祭でアニメとして初めて金熊賞を獲得した、宮崎駿監督の映画は何でしょう？
■ 出力:
『となりのトトロ』
■ 正解ラベル:
千と千尋の神隠し


■ 入力:
英語で「過失」という意味がある、テニスでサーブがサービスコートに正しく入らないで失敗することを何というでしょう？
■ 出力:
フォア
■ 正解ラベル:
フォ（ー）ルト


■ 入力:
もともとはカトリック教徒が使っていた、大小の玉を連ねて十字架をつけたネックレス状のアクセサリーといえば何でしょう？
■ 出力:
ロザリオ
■ 正解ラベル:
ロザリオ


